In [ ]:
# Qiwei Li
# https://www.linkedin.com/in/liqiwei
%matplotlib inline

from datetime import datetime
from collections import defaultdict
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import timeit
from sys import stdout

In [ ]:
checkInDensityByHotelMarket = pickle.load(open("data/checkInDensityByHotelMarket.p", "rb"))
checkOutDensityByHotelMarket = pickle.load(open("data/checkOutDensityByHotelMarket.p", "rb"))
trainDataFrameRowByDestId = pickle.load(open("data/trainDataFrameRowByDestId.p", "rb"))
testDataFrameRowByDestId = pickle.load(open("data/trainDataFrameRowByDestId.p", "rb"))

In [ ]:
trainDataFrameByDestId = dict()
testDataFrameByDestId = dict()
trainFillCounter = dict()
testFillCounter = dict()

In [ ]:
colNames = ('hour','dow','posa','dist','mobile','package','channel','ci','co',
            'adults','children','room','stay','type','booking','cluster')

print('Preallocating space ... ' + str(datetime.now().time()))

for key, value in trainDataFrameRowByDestId.iteritems():
    df = pd.DataFrame(index=np.arange(0, value), columns=colNames)
    trainDataFrameByDestId[key] = df
    trainFillCounter[key] = 0

print('Done ... ' + str(datetime.now().time()))

In [ ]:
print('Opening train ... ' + str(datetime.now().time()))
f = open("data/train.csv", "r")
f.readline()
total = 0

while 1:
    line = f.readline().strip()
    total += 1
        
    stdout.write("\r%d" % total)
    stdout.flush()
    
    if line == '':
         break
            
    line = f.readline().strip()
    arr = line.split(",")
    
    if arr[16] == '' or arr[21] == '' or arr[22] == '':
        continue
        
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]
    
    try:
        search_time_DT = datetime.strptime(arr[0], '%Y-%m-%d %H:%M:%S')
        arr[0] = search_time_DT.hour
        arr[1] = search_time_DT.weekday()

        checkInDateDT = datetime.strptime(arr[11], "%Y-%m-%d")
        checkOutDateDT = datetime.strptime(arr[12], "%Y-%m-%d")
        arr[11] = checkInDateDT.timetuple().tm_yday
        arr[12] = checkOutDateDT.timetuple().tm_yday
    
        arr[16] = arr[12] - arr[11]
    
        arr[11] = checkInDensityByHotelMarket[hotel_country, hotel_market](arr[11])
        arr[12] = checkOutDensityByHotelMarket[hotel_country, hotel_market](arr[12])
    except:
        continue
        
    useless = [22,21,20,19,7,5,4,3]
    for i in useless:
        del arr[i]
    
    current = trainFillCounter[(hotel_market, srch_destination_id)]
    trainDataFrameByDestId[(hotel_market, srch_destination_id)].loc[current,] = arr
    trainFillCounter[(hotel_market, srch_destination_id)] += 1

print('Closing train ... ' + str(datetime.now().time()))
f.close()